In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('presupuestos_comunas_anios_comma.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   comuna       135 non-null    int64  
 1   desc_comuna  135 non-null    object 
 2   presupuesto  135 non-null    float64
 3   anio         135 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 4.3+ KB


In [8]:
df.head(5)

,comuna,desc_comuna,presupuesto,anio
0,1,COMUNA 1,3.414951e+10,2013
1,2,COMUNA 2,2.794259e+09,2013
2,3,COMUNA 3,5.588101e+09,2013
3,4,COMUNA 4,2.057754e+10,2013
4,5,COMUNA 5,3.904953e+09,2013


#### Primero vamos a hacer un promedio y ver el nivel en cada comuna

In [10]:
import pandas as pd
import pymysql
import geopandas as gpd
from shapely import wkt
import folium
import branca
import branca.colormap as cm

secciones = gpd.read_file('comunas_wgs84.zip')
secciones = secciones[['COMUNAS','geometry']]
secciones = secciones.rename(columns={'COMUNAS':'comuna'})
secciones['comuna'] = secciones['comuna'].astype(int)
secciones['comuna'] = secciones['comuna'].astype(str).str.zfill(2)
# secciones['geometry'] = secciones['geometry'].apply(wkt.loads)
secciones.crs = 'EPSG:4326'
secciones


,comuna,geometry
0,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5..."
1,06,"POLYGON ((-58.43061 -34.60705, -58.43056 -34.6..."
2,10,"POLYGON ((-58.48834 -34.62016, -58.48820 -34.6..."
3,11,"POLYGON ((-58.49838 -34.59637, -58.49718 -34.5..."
4,12,"POLYGON ((-58.50331 -34.59386, -58.50347 -34.5..."
5,14,"POLYGON ((-58.42676 -34.55202, -58.42676 -34.5..."
6,15,"POLYGON ((-58.42602 -34.59661, -58.42484 -34.5..."
7,13,"POLYGON ((-58.44033 -34.54022, -58.44032 -34.5..."
8,04,"POLYGON ((-58.38864 -34.63373, -58.38864 -34.6..."
9,05,"POLYGON ((-58.43333 -34.60267, -58.43314 -34.6..."


In [12]:
df['comuna'] = df['comuna'].astype(str).str.zfill(2)
df.head()

,comuna,desc_comuna,presupuesto,anio
0,01,COMUNA 1,3.414951e+10,2013
1,02,COMUNA 2,2.794259e+09,2013
2,03,COMUNA 3,5.588101e+09,2013
3,04,COMUNA 4,2.057754e+10,2013
4,05,COMUNA 5,3.904953e+09,2013


In [14]:
df_presupuesto = secciones.merge(df, on = 'comuna', how='inner')
df_presupuesto


,comuna,geometry,desc_comuna,presupuesto,anio
0,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,2.794259e+09,2013
1,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,4.550369e+09,2014
2,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,7.967680e+09,2015
3,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,1.250950e+10,2016
4,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,1.967894e+10,2017
...,...,...,...,...,...
130,01,"MULTIPOLYGON (((-58.36854 -34.57365, -58.36852...",COMUNA 1,2.441865e+11,2017
131,01,"MULTIPOLYGON (((-58.36854 -34.57365, -58.36852...",COMUNA 1,3.057346e+11,2018
132,01,"MULTIPOLYGON (((-58.36854 -34.57365, -58.36852...",COMUNA 1,4.767567e+11,2019
133,01,"MULTIPOLYGON (((-58.36854 -34.57365, -58.36852...",COMUNA 1,7.003267e+11,2020


In [16]:
df_presupuesto = df_presupuesto[df_presupuesto['anio'] == 2013]

In [17]:
#HOVER COLOR
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

#NSE FUNCION COLOR
#CAPA ELECTORES

#escala
escala_de_color = df_presupuesto.copy() #hagamos una copia de circuitos
vals = escala_de_color['presupuesto'].describe()

#ordeno los numeros (vals) para generar la escala de colores
if vals['min'] == 0:
    ordenada = [vals['25%']-vals['std'],vals['25%'],vals['mean'],vals['50%'],vals['mean']+vals['std'],vals['75%'],vals['max']]
    ordenada = sorted(ordenada)
else:
    ordenada = [vals['min'],vals['25%'],vals['mean'],vals['50%'],vals['mean']+vals['std'],vals['75%'],vals['max']]
    ordenada = sorted(ordenada)
for  num,x in enumerate(ordenada):
    try:
        if x <= 0 and ordenada[num+1] > 0:
            ordenada[num+1] = 0
            break
    except:
        pass
    
#ordenada

#traigo los colores que voy a necesitar
positivos = ['#FFF176','#d8ff65','#bbe27e','#9bd467','#70bf5a','#45ab4d','#1a9641'] 
colores = positivos

#Armamos la escala de colores pasando por parametros(colors,index y los extremos (vmin,vmax))
colormaplinear = cm.LinearColormap(colors=colores, index=ordenada,vmin=min(ordenada),vmax=max(ordenada))

#Para ver que devuelve cada valor
colormaplinear(100000), colormaplinear(0), colormaplinear(700000)

#funcion que nos devuelva automaticamente los colores que queremos dependiendo de los valores para electores que tenemos en el dataframe

scores_series = df_presupuesto['presupuesto'] #tomamos la columna de electores

def my_color_function(feature):
    #obtenemos el id del la "caracteristica" que queremos para pasar el color a cada poligono
    #esto lo vamos a ver en detalle mas adelante cuando vayamos a agregar la capa de electores
    
    scores = scores_series.get(int(feature['id']), None) 
    
    # aca si el circuito es nulo, devuelvo un color gris que es este '#8c8c8c y sino utiliza nuestra
    # HERMOSA escala de colores que creamos anteriormente
    return '#8c8c8c' if (pd.isnull(scores)) else colormaplinear(scores)



In [18]:
df_presupuesto.head()

,comuna,geometry,desc_comuna,presupuesto,anio
0,02,"POLYGON ((-58.38000 -34.57002, -58.38002 -34.5...",COMUNA 2,2.794259e+09,2013
9,06,"POLYGON ((-58.43061 -34.60705, -58.43056 -34.6...",COMUNA 6,3.959265e+09,2013
18,10,"POLYGON ((-58.48834 -34.62016, -58.48820 -34.6...",COMUNA 10,2.717458e+09,2013
27,11,"POLYGON ((-58.49838 -34.59637, -58.49718 -34.5...",COMUNA 11,3.070007e+09,2013
36,12,"POLYGON ((-58.50331 -34.59386, -58.50347 -34.5...",COMUNA 12,3.561739e+09,2013


In [27]:
c_3 = folium.FeatureGroup(name="Presupuesto por comuna 2013", show=True) #show false hace que por defecto no se muestre

c_3.add_child(folium.GeoJson(
                df_presupuesto[['geometry','desc_comuna','presupuesto']], #de nuevo las columnas que vamos a usar
                highlight_function=highlight_function, #esta funcion nos permite que al pasar el mouse se ponga en otro color
                style_function=lambda feature: { #aca usamos una funcion lambda para cada "linea" de dataset
                                                 #automaticamente folium mapea/itera en cada una de las filas para 
                                                 #agregar lo que queremos
                'color':'#000000', #el color de la linea de contorno
                'fillOpacity': 0.75, #la opacidad de los colores de relleno
                'weight': 1, #el peso de la linea
                'fillColor': my_color_function(feature), #usamos nuestra funcion de colores
            },
                # seteamos los datos que se quieren mostrar el en tooltip
               tooltip=folium.features.GeoJsonTooltip(
                    fields= ['desc_comuna','presupuesto'], #ponemos que columnas vamos a mostrar
                    aliases = ['Comuna','Presupuesto'], #ponemos los nombres con los que se van a ver
                    labels=True,
                    sticky=True,
                    localize=True)

            ))

In [28]:
generador_centro_localidad = (i for i in df_presupuesto['geometry'].centroid) 

#defino otro generador para las etiquetas
generador_etiqueta_localidad = (str(i).replace('nan',' Sin Datos.').replace('.0','')  for i in df_presupuesto['presupuesto'].values)

#viene la magia de agregar a la capa_2 la etiqueta que corresponde, aca a pensar un poco, si explico todo temrino en 1 mes
for c,e in zip(generador_centro_localidad,generador_etiqueta_localidad):
    try :
        etiqueta_loc = str(format(int(e),",d"))
    except:
        etiqueta_loc = e
    c_3.add_child(folium.Marker(
    location=[c.y,c.x],
    popup=etiqueta_loc,
    icon=folium.DivIcon(html=f"""<div style="font-family: Trebuchet MS; font-size: 13px; color: black;">{etiqueta_loc}</div>""")
    ))

C:\Users\Usuario\anaconda3\envs\work_env\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [29]:
m = folium.Map(location=[-34.60302806157693, -58.447040437773566], tiles = 'cartodbpositron', zoom_start=12, control_scale=True)

m.add_child(c_3)

colormaplinear.caption = "Presupuesto 2013 por comuna" #Titulo para la escala de colores

m.add_child(colormaplinear)
folium.LayerControl().add_to(m)

m


In [50]:
m.save('NSE_CABA.html')